#Transformiered des Datenset um DAG-Variablen representieren zu können

In [5]:
import pandas as pd
from scipy.stats import truncnorm
from IPython.display import display

In [6]:
path_Manipulated = r"RealDataMerged\mergedBased.xlsx"

In [7]:
df_Manipulated = pd.read_excel(path_Manipulated)
display(df_Manipulated.head(5))

,FANummer,Menge,Startdatum,Enddatum,Fälligkeitsdatum,Fertigstellungsdatum,OperationNo,Logdatum,DurchlaufszeitIst(h),DurchlaufszeitIst(t),Rüst-Bearbeitungszeit(h),Rüstzeit(h),Bearbeitungszeit(h)
0,FA1112305,21,2024-06-12,2024-06-21,2024-06-21,2024-06-26,15,2024-06-13 14:49:01,117.59,14.70,17.12,0.45,16.67
1,FA1114090,300,2024-05-17,2024-05-30,2024-05-30,2024-05-31,70,2024-05-30 11:29:25,0.01,0.00,0.03,0.02,0.02
2,FA1114511,36,2024-05-16,2024-05-29,2024-05-29,2024-05-29,90,2024-05-28 11:59:28,0.00,0.00,3.07,0.33,2.73
3,FA1114659,50,2024-05-06,2024-05-31,2024-05-31,2024-05-31,50,2024-05-29 14:05:46,8.48,1.06,6.85,1.88,4.97
4,FA1114659,50,2024-05-06,2024-05-31,2024-05-31,2024-05-31,60,2024-05-29 14:06:41,0.02,0.00,0.00,0.00,0.00


In [8]:
df_Manipulated['Startdatum'] = pd.to_datetime(df_Manipulated['Startdatum'])
df_Manipulated['Enddatum'] = pd.to_datetime(df_Manipulated['Enddatum'])

# Neue Spalte mit Differenz in Tagen --> entspricht Durchlaufzeit Soll
df_Manipulated['DurchlaufszeitSoll(t)'] = (df_Manipulated['Enddatum'] - df_Manipulated['Startdatum']).dt.days
display(df_Manipulated.head(3))

,FANummer,Menge,Startdatum,Enddatum,Fälligkeitsdatum,Fertigstellungsdatum,OperationNo,Logdatum,DurchlaufszeitIst(h),DurchlaufszeitIst(t),Rüst-Bearbeitungszeit(h),Rüstzeit(h),Bearbeitungszeit(h),DurchlaufszeitSoll(t)
0,FA1112305,21,2024-06-12,2024-06-21,2024-06-21,2024-06-26,15,2024-06-13 14:49:01,117.59,14.7,17.12,0.45,16.67,9
1,FA1114090,300,2024-05-17,2024-05-30,2024-05-30,2024-05-31,70,2024-05-30 11:29:25,0.01,0.0,0.03,0.02,0.02,13
2,FA1114511,36,2024-05-16,2024-05-29,2024-05-29,2024-05-29,90,2024-05-28 11:59:28,0.00,0.0,3.07,0.33,2.73,13


In [9]:
df_Manipulated['Soll/Ist Durchlaufszeit(t)'] = (df_Manipulated['DurchlaufszeitSoll(t)'] - df_Manipulated['DurchlaufszeitIst(t)'])
display(df_Manipulated.head(3))

,FANummer,Menge,Startdatum,Enddatum,Fälligkeitsdatum,Fertigstellungsdatum,OperationNo,Logdatum,DurchlaufszeitIst(h),DurchlaufszeitIst(t),Rüst-Bearbeitungszeit(h),Rüstzeit(h),Bearbeitungszeit(h),DurchlaufszeitSoll(t),Soll/Ist Durchlaufszeit(t)
0,FA1112305,21,2024-06-12,2024-06-21,2024-06-21,2024-06-26,15,2024-06-13 14:49:01,117.59,14.7,17.12,0.45,16.67,9,-5.7
1,FA1114090,300,2024-05-17,2024-05-30,2024-05-30,2024-05-31,70,2024-05-30 11:29:25,0.01,0.0,0.03,0.02,0.02,13,13.0
2,FA1114511,36,2024-05-16,2024-05-29,2024-05-29,2024-05-29,90,2024-05-28 11:59:28,0.00,0.0,3.07,0.33,2.73,13,13.0


In [10]:
# Stelle sicher, dass beide Spalten im Datetime-Format sind
df_Manipulated['Fälligkeitsdatum'] = pd.to_datetime(df_Manipulated['Fälligkeitsdatum'])
df_Manipulated['Fertigstellungsdatum'] = pd.to_datetime(df_Manipulated['Fertigstellungsdatum'])

# Neue Spalte mit Abweichung in Tagen (positiv = verspätet, negativ = zu früh)
df_Manipulated['Abweichung (Tage)'] = (df_Manipulated['Fertigstellungsdatum'] - df_Manipulated['Fälligkeitsdatum']).dt.days

# Neue Spalte zur Bewertung der Termintreue
df_Manipulated['Termintreue'] = df_Manipulated['Abweichung (Tage)'].apply(
    lambda x: 0 if x <= 0 else 1)

display(df_Manipulated.head(3))

,FANummer,Menge,Startdatum,Enddatum,Fälligkeitsdatum,Fertigstellungsdatum,OperationNo,Logdatum,DurchlaufszeitIst(h),DurchlaufszeitIst(t),Rüst-Bearbeitungszeit(h),Rüstzeit(h),Bearbeitungszeit(h),DurchlaufszeitSoll(t),Soll/Ist Durchlaufszeit(t),Abweichung (Tage),Termintreue
0,FA1112305,21,2024-06-12,2024-06-21,2024-06-21,2024-06-26,15,2024-06-13 14:49:01,117.59,14.7,17.12,0.45,16.67,9,-5.7,5,1
1,FA1114090,300,2024-05-17,2024-05-30,2024-05-30,2024-05-31,70,2024-05-30 11:29:25,0.01,0.0,0.03,0.02,0.02,13,13.0,1,1
2,FA1114511,36,2024-05-16,2024-05-29,2024-05-29,2024-05-29,90,2024-05-28 11:59:28,0.00,0.0,3.07,0.33,2.73,13,13.0,0,0


In [11]:
df_Manipulated.drop(columns=['Startdatum', 'Enddatum', 'Fälligkeitsdatum', 'Fertigstellungsdatum', 'Logdatum', 'DurchlaufszeitIst(h)'], inplace=True)
display(df_Manipulated.head(5))

,FANummer,Menge,OperationNo,DurchlaufszeitIst(t),Rüst-Bearbeitungszeit(h),Rüstzeit(h),Bearbeitungszeit(h),DurchlaufszeitSoll(t),Soll/Ist Durchlaufszeit(t),Abweichung (Tage),Termintreue
0,FA1112305,21,15,14.70,17.12,0.45,16.67,9,-5.70,5,1
1,FA1114090,300,70,0.00,0.03,0.02,0.02,13,13.00,1,1
2,FA1114511,36,90,0.00,3.07,0.33,2.73,13,13.00,0,0
3,FA1114659,50,50,1.06,6.85,1.88,4.97,25,23.94,0,0
4,FA1114659,50,60,0.00,0.00,0.00,0.00,25,25.00,0,0


In [12]:
df_Manipulated.describe(include='all')

,FANummer,Menge,OperationNo,DurchlaufszeitIst(t),Rüst-Bearbeitungszeit(h),Rüstzeit(h),Bearbeitungszeit(h),DurchlaufszeitSoll(t),Soll/Ist Durchlaufszeit(t),Abweichung (Tage),Termintreue
count,2897,2897.000000,2897.000000,2897.000000,2897.000000,2897.000000,2897.000000,2897.000000,2897.000000,2897.000000,2897.000000
unique,1468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,FA1116759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,29.617881,74.161892,1.355830,1.291878,0.323738,0.968488,7.493614,6.137784,4.067311,0.657922
std,NaN,64.978922,240.291904,2.256516,2.515336,0.331333,2.405681,15.526267,15.553265,9.641142,0.474487
min,NaN,1.000000,10.000000,0.000000,-16.800000,-1.670000,-16.800000,0.000000,-22.170000,-41.000000,0.000000
25%,NaN,2.000000,20.000000,0.000000,0.320000,0.120000,0.050000,4.000000,2.000000,0.000000,0.000000
50%,NaN,10.000000,30.000000,0.530000,0.580000,0.300000,0.250000,6.000000,4.850000,2.000000,1.000000
75%,NaN,25.000000,40.000000,2.000000,1.270000,0.420000,0.830000,8.000000,7.000000,7.000000,1.000000


In [13]:
cols_merged = [
    'Menge',
    'DurchlaufszeitIst(t)',
    'Rüst-Bearbeitungszeit(h)',
    'Rüstzeit(h)',
    'Bearbeitungszeit(h)',
    'DurchlaufszeitSoll(t)',
    'Soll/Ist Durchlaufszeit(t)',
    'Abweichung (Tage)'
]

corr_matrix = df_Manipulated[cols_merged].corr()
cov_matrix  = df_Manipulated[cols_merged].cov()

print("Korrelationsmatrix:")
display(corr_matrix.round(3))
print("\nKovarianzmatrix:")
display(cov_matrix.round(3))


Korrelationsmatrix:


,Menge,DurchlaufszeitIst(t),Rüst-Bearbeitungszeit(h),Rüstzeit(h),Bearbeitungszeit(h),DurchlaufszeitSoll(t),Soll/Ist Durchlaufszeit(t),Abweichung (Tage)
Menge,1.000,0.133,0.703,0.065,0.726,0.154,0.134,0.054
DurchlaufszeitIst(t),0.133,1.000,0.269,0.209,0.252,0.061,-0.084,0.060
Rüst-Bearbeitungszeit(h),0.703,0.269,1.000,0.389,0.992,0.144,0.105,0.034
Rüstzeit(h),0.065,0.209,0.389,1.000,0.269,0.080,0.050,0.015
Bearbeitungszeit(h),0.726,0.252,0.992,0.269,1.000,0.140,0.103,0.034
DurchlaufszeitSoll(t),0.154,0.061,0.144,0.080,0.140,1.000,0.989,0.026
Soll/Ist Durchlaufszeit(t),0.134,-0.084,0.105,0.050,0.103,0.989,1.000,0.018
Abweichung (Tage),0.054,0.060,0.034,0.015,0.034,0.026,0.018,1.000



Kovarianzmatrix:


,Menge,DurchlaufszeitIst(t),Rüst-Bearbeitungszeit(h),Rüstzeit(h),Bearbeitungszeit(h),DurchlaufszeitSoll(t),Soll/Ist Durchlaufszeit(t),Abweichung (Tage)
Menge,4222.260,19.551,114.953,1.400,113.558,155.196,135.644,33.763
DurchlaufszeitIst(t),19.551,5.092,1.527,0.157,1.370,2.126,-2.965,1.304
Rüst-Bearbeitungszeit(h),114.953,1.527,6.327,0.324,6.002,5.633,4.106,0.827
Rüstzeit(h),1.400,0.157,0.324,0.110,0.214,0.413,0.256,0.049
Bearbeitungszeit(h),113.558,1.370,6.002,0.214,5.787,5.219,3.849,0.777
DurchlaufszeitSoll(t),155.196,2.126,5.633,0.413,5.219,241.065,238.939,3.940
Soll/Ist Durchlaufszeit(t),135.644,-2.965,4.106,0.256,3.849,238.939,241.904,2.636
Abweichung (Tage),33.763,1.304,0.827,0.049,0.777,3.940,2.636,92.952


In [ ]:
#Speichert manipuliertes Datenset

output_xlsx = r"RealDataMerged/mergedManupulated_2.xlsx"
df_Manipulated.to_excel(output_xlsx, index=False)